<a href="https://colab.research.google.com/github/akul-bharadwaj/Welmart-sales-insights-with-PySpark/blob/main/Welmart_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Superstore Sales Insights

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.appName('Superstore_Sales').getOrCreate()

### Data Loading

In [ ]:
# Path to data set
csv_file = "/content/Superstore.csv"

df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(csv_file)

In [ ]:
df.createOrReplaceTempView("u")

In [ ]:
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|   

### 1. Determine the best-selling product sub-category.

In [ ]:
#Determine the best-selling product sub-category.

sales_by_sub_category = df.groupBy("Sub-Category").agg(sum("Sales").alias("Total_Sales"))

In [ ]:
sales_by_sub_category.orderBy("Total_Sales", ascending=False).show()

+------------+------------------+
|Sub-Category|       Total_Sales|
+------------+------------------+
|      Phones| 329753.0880000001|
|      Chairs|328449.10300000076|
|     Storage|216803.21200000012|
|      Tables| 206965.5320000001|
|     Binders|199905.71700000006|
|    Machines|189238.63099999996|
| Accessories| 167380.3180000001|
|     Copiers|149528.02999999994|
|   Bookcases|114879.99629999997|
|  Appliances|        107532.161|
| Furnishings| 82752.23000000004|
|       Paper| 75356.11799999999|
|    Supplies| 45952.47000000001|
|         Art|27118.791999999954|
|   Envelopes|15339.489999999993|
|      Labels|         12486.312|
|   Fasteners|3008.6559999999995|
+------------+------------------+



In [ ]:
best_selling_sub_category = sales_by_sub_category.orderBy("Total_Sales", ascending=False).limit(1)
best_selling_sub_category.show()

+------------+-----------------+
|Sub-Category|      Total_Sales|
+------------+-----------------+
|      Phones|329753.0880000001|
+------------+-----------------+



### 2. Identify the product category generating the highest revenue.

In [ ]:
#Identify the product category generating the highest revenue.

rev_by_category = df.groupBy("Category").agg(sum("Sales").alias("Total_Sales"))

In [ ]:
rev_by_category.orderBy("Total_Sales", ascending=False).show()

+---------------+-----------------+
|       Category|      Total_Sales|
+---------------+-----------------+
|     Technology|835900.0669999964|
|      Furniture|733046.8612999996|
|Office Supplies|703502.9280000031|
+---------------+-----------------+



In [ ]:
highest_rev_category = rev_by_category.orderBy("Total_Sales", ascending=False).limit(1)
highest_rev_category.show()

+----------+-----------------+
|  Category|      Total_Sales|
+----------+-----------------+
|Technology|835900.0669999964|
+----------+-----------------+



### 3. Compile a top 10 list of the most valuable customers.

In [ ]:
#Compile a top 10 list of the most valuable customers.

cust_rev = df.groupBy("Customer Name").agg(sum("Sales").alias("Total_Sales"))

In [ ]:
cust_rev.orderBy("Total_Sales", ascending=False).show(10)

+------------------+------------------+
|     Customer Name|       Total_Sales|
+------------------+------------------+
|       Sean Miller|          25043.05|
|      Tamara Chand|19017.847999999998|
|      Raymond Buch|         15117.339|
|      Tom Ashbrook|          14595.62|
|     Adrian Barton|14355.610999999997|
|      Sanjit Chand|14142.333999999999|
|      Ken Lonsdale|         14071.917|
|      Hunter Lopez|12873.297999999999|
|      Sanjit Engle|12209.438000000002|
|Christopher Conant|         12129.072|
+------------------+------------------+
only showing top 10 rows



### 4. Determine the state responsible for the highest number of orders.

In [ ]:
#Determine the state responsible for the highest number of orders.

state_orders = df.groupBy("State").agg(count("*").alias("Total_Orders"))

In [ ]:
state_orders.orderBy("Total_Orders", ascending=False).show()

+--------------+------------+
|         State|Total_Orders|
+--------------+------------+
|    California|        2001|
|      New York|        1128|
|         Texas|         985|
|  Pennsylvania|         587|
|    Washington|         506|
|      Illinois|         492|
|          Ohio|         469|
|       Florida|         383|
|      Michigan|         255|
|North Carolina|         249|
|      Virginia|         224|
|       Arizona|         224|
|       Georgia|         184|
|     Tennessee|         183|
|      Colorado|         182|
|       Indiana|         149|
|      Kentucky|         139|
| Massachusetts|         135|
|    New Jersey|         130|
|        Oregon|         124|
+--------------+------------+
only showing top 20 rows



In [ ]:
state_orders.orderBy("Total_Orders", ascending=False).show(1)

+----------+------------+
|     State|Total_Orders|
+----------+------------+
|California|        2001|
+----------+------------+
only showing top 1 row



### 5. Find the year with the highest revenue generation.

In [ ]:
# Find the year with the highest revenue generation.

#df = df.withColumn("Order Date", to_date(df["Order Date"], "MM/dd/yyyy"))
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

rev_by_year = df.groupBy(year("Order Date").alias("Year")).agg(sum("Sales").alias("Total_Sales"))

In [ ]:
rev_by_year.orderBy("Total_Sales", ascending=False).show()

+----+-----------------+
|Year|      Total_Sales|
+----+-----------------+
|2017|724994.5611999998|
|2016|601265.2610000009|
|2014| 481763.798100001|
|2015|464426.2359999999|
+----+-----------------+



In [ ]:
rev_by_year.orderBy("Total_Sales", ascending=False).show(1)

+----+-----------------+
|Year|      Total_Sales|
+----+-----------------+
|2017|724994.5611999998|
+----+-----------------+
only showing top 1 row

